# Task 2: Data Exploration and Processing

## 1. Manual data inspection
- Investigate which standard and potential new NER types are most prominent in your data set (i.e., manual data inspection)

Following visual inspection,some of the prominent NERs found are: DATE, BODY_PART, DOSAGE, MEASUREMENT, DRUG and SYMPTOM. 

Examples:
 1. DATE: 12/20/2005, 1/19/96
 2. BODY_PART: nose, abdomen, knee
 3. DOSAGE:  10/40 mg one a day, 0.25 micrograms a day, 50 mg twice a day, 10 ml
 4. MEASUREMENT: 3.98 kg, 8mm, pulse of 84, blood pressure 108/65
 5. DRUG:  Vytorin, Rocaltrol, Carvedilol, Cozaar,  Lasix
 6. SYMPTON: erythematous, chest pain, constipated

Where, DATE is a **standard NER in spacy** and the remaining ones fall in the medicine domain category. 

## 2. Apply the standard NER classifier of spaCy


### Imports

In [1]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import spacy
import random



c:\Users\carla\.conda\envs\llm-dev\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
c:\Users\carla\.conda\envs\llm-dev\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Data

In [2]:
# ============================
# 1: Load Dataset
# ============================
dataset = load_dataset("argilla/medical-domain", split="train")
print(len(dataset))

4966


In [3]:
# ================================================
# 2: Define Entity Schema
# ================================================
# Our final gold-label schema
CUSTOM_LABELS = [

    "DISEASE", "BODY_PART", "PROCEDURE", "FINDING", "SYMPTOM"
]

print("Our NER schema = ", CUSTOM_LABELS)


Our NER schema =  ['DISEASE', 'BODY_PART', 'PROCEDURE', 'FINDING', 'SYMPTOM']


In [4]:
# =========================================
# 3: Pick N samples manually
# =========================================
PATH_TO_ANNOTATIONS = "ner/samples/"
SEED = 42 
random.seed(SEED)
np.random.seed(SEED)

N_SAMPLES = 2   # select a proper number to contain 100 ground-truth NERs 
sampled_texts = []

for i in range(N_SAMPLES):
    row = dataset[np.random.randint(0, len(dataset))]
    text = row["text"] if "text" in row else row["content"]
    sent = row["text"] if i==0 else None
    sampled_texts.append(text)

df_samples = pd.DataFrame({"text": sampled_texts})
df_samples.to_csv(PATH_TO_ANNOTATIONS + "sample_for_annotation.csv", index=False)

print("Saved sample_for_annotation.csv with", len(df_samples), "sentences")


Saved sample_for_annotation.csv with 2 sentences


## 3. Evaluation of Standard NER


### Generate Templates for Manual Annotation. 

 The annotation format should be: 

    {
        "sample_id": 0, 
        "sentence_id": 0, 
        "text": "REASON FOR THE CONSULT:,  Nonhealing right ankle stasis ulcer.,HISTORY", 
        "annotation": "[["Nonhealing","FINDING"],["right ankle","BODY_PART"],["stasis ulcer","DISEASE"]]"
    }



In [5]:
# ============================================
# Generate train/test annotation template JSONL with ratio control
# ============================================
INPUT_CSV = PATH_TO_ANNOTATIONS + "sample_for_annotation.csv"

# ================================
# Parameters: control split ratio
# ================================
TRAIN_RATIO = 0.70     # first 70% for training（sample 0）
TEST_RATIO  = 0.30     # first 30% for evaluation/test（sample 1）

# ================================

# 1. Load samples: only column "text"
df = pd.read_csv(INPUT_CSV)
df["sample_id"] = df.index  # auto-index

# 2. Load spaCy for sentence splitting
nlp = spacy.load("en_core_web_md")
if "sentencizer" not in nlp.pipe_names:
    nlp.add_pipe("sentencizer")

# Buckets
train_items = []
test_items = []

# ===========================
# Split sentences per sample
# ===========================
for _, row in df.iterrows():

    sample_id = int(row["sample_id"])
    full_text = str(row["text"])
    doc = nlp(full_text)

    # Extract sentences
    sents = [s.text.strip() for s in doc.sents if len(s.text.strip()) > 0]
    total = len(sents)

    if sample_id == 0:
        # sample 0 → train sample
        cutoff = int(total * TRAIN_RATIO)
        selected = sents[:cutoff]

        for i, text in enumerate(selected):
            train_items.append({
                "sample_id": sample_id,
                "sentence_id": i,
                "text": text,
                "annotation": []
            })

    elif sample_id == 1:
        # sample 1 → test sample
        cutoff = int(total * TEST_RATIO)
        selected = sents[:cutoff]

        for i, text in enumerate(selected):
            test_items.append({
                "sample_id": sample_id,
                "sentence_id": i,
                "text": text,
                "annotation": []
            })

print("Sample 0 sentences:", len([s for s in nlp(df.loc[0, "text"]).sents]))
print("Sample 1 sentences:", len([s for s in nlp(df.loc[1, "text"]).sents]))



Sample 0 sentences: 83
Sample 1 sentences: 89


### 3.1 Manual Evaluation of Standard NER

In [6]:
### Manual Evaluation of Standard NER

import spacy
import pandas as pd
from spacy import displacy

# 1. Load sample text file
df_samples = pd.read_csv(PATH_TO_ANNOTATIONS + "sample_for_annotation.csv")

# 2. Extract second row text
# sample_for_annotation.csv has:
# row 0: header "text"
# row 1: sample 1 -> this is the train sample for further NER extention 
# row 2: sample 2  -> this is the test sample
test_text = df_samples.iloc[1]["text"]

print("=== Test Text Preview ===")

# 3. Load spaCy model (baseline or your updated one)
nlp = spacy.load("en_core_web_md")   # or: spacy.load("output_medical_ner")

# 4. Run NER on the full document
doc = nlp(test_text[:2000])

# 5. Render using displacy
displacy.render(doc, style="ent", jupyter=True)


=== Test Text Preview ===


Observation: Some NERs don't make sense. E.g., Vomitting -> PERSON, Hypokalemia -> PERSON, Diarrhea -> PERSON

### 3.2 Automatic Evaluation of Standard NER

In [7]:
# =========================================
#  Standard NER evaluation
# =========================================

import json
import spacy
from sklearn.metrics import precision_recall_fscore_support
from ner.utils import extract_spans, convert_to_labels

# 1. Load JSONL annotations
jsonl_path = PATH_TO_ANNOTATIONS + "test_annotation_complete.jsonl"

data = []
with open(jsonl_path, "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        data.append(obj)

print(f"Loaded {len(data)} annotated sentences")

# 2. Load baseline spaCy NER
nlp = spacy.load("en_core_web_md")

# 3. Convert annotations to character-level spans
gold_spans, pred_spans, labels = extract_spans(nlp, data)

# 4. Convert spans to entity sets for evaluation
true_labels, pred_labels = convert_to_labels(gold_spans, pred_spans)

# 5. Evaluate macro and micro F1
prec, rec, f1, _ = precision_recall_fscore_support(
    true_labels, pred_labels, average="micro", zero_division=0
)

print("\n===== Baseline spaCy NER Evaluation =====")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 score:  {f1:.4f}")

prec_m, rec_m, f1_m, _ = precision_recall_fscore_support(
    true_labels, pred_labels, average="macro", zero_division=0
)

print("\nMacro Precision:", round(prec_m, 4))
print("Macro Recall:   ", round(rec_m, 4))
print("Macro F1:       ", round(f1_m, 4))

# 6. Show some error cases
print("\n===== Examples of WRONG predictions =====\n")

for i, (g, p, item) in enumerate(zip(gold_spans, pred_spans, data)):
    g_set = set(g)
    p_set = set(p)
    if g_set != p_set:
        print("Text:", item["text"])
        print("Gold:", g_set)
        print("Pred:", p_set)
        print("-" * 50)
        break

Loaded 26 annotated sentences

===== Baseline spaCy NER Evaluation =====
Precision: 0.0000
Recall:    0.0000
F1 score:  0.0000

Macro Precision: 0.0
Macro Recall:    0.0
Macro F1:        0.0

===== Examples of WRONG predictions =====

Text: ADMISSION DIAGNOSES:,1.
Gold: set()
Pred: {(0, 9, 'ORG')}
--------------------------------------------------


## 4. Extend the standard NER types using the NER Annotator

### 4.1 Training Extended NER model with >100 manual annoatation

In [8]:
# Make sure there is no overlapping entity problems in the training sample
# 
# import json

path = PATH_TO_ANNOTATIONS + "train_annotation_complete.jsonl"

def spans_overlap(a, b):
    return not (a[1] <= b[0] or b[1] <= a[0])

bad = []

with open(path, "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        text = obj["text"]
        anns = obj["annotation"]  # [[phrase,label],...]

        spans = []
        for phrase, label in anns:
            start = text.lower().find(phrase.lower())
            if start == -1:
                continue
            end = start + len(phrase)
            spans.append((start, end, phrase, label))

        # check overlapping
        for i in range(len(spans)):
            for j in range(i+1, len(spans)):
                a = spans[i]
                b = spans[j]
                if spans_overlap(a, b):
                    bad.append((obj["sentence_id"], text, a, b))

print("===== Overlapping entity problems found =====")
for sid, text, a, b in bad:
    print(f"\nSentence {sid}: {text}")
    print("  ->", a)
    print("  ->", b)


===== Overlapping entity problems found =====


In [9]:
# =========================================
#  Extended NER TRAINING
# =========================================

import json
import random
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding
from ner.utils import find_span

# 1. Load annotated JSONL
json_path = PATH_TO_ANNOTATIONS + "train_annotation_complete.jsonl"
data = []

with open(json_path, "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))

print(f"Loaded {len(data)} annotated sentences")

# 2. Custom labels
CUSTOM_LABELS = ["DISEASE", "BODY_PART", "FINDING", "PROCEDURE", "SYMPTOM"]
print("Custom NER labels:", CUSTOM_LABELS)

# 3. Prepare training data
training_examples = []

for item in data:
    text = item["text"]
    labels = item["annotation"]  # [["hypertension","DISEASE"], ...]

    entities = []
    for phrase, label in labels:
        span = find_span(text, phrase)
        if span:
            entities.append((span[0], span[1], label))

    training_examples.append((text, {"entities": entities}))

print("Example:", training_examples[0])

# 4. Initialize blank model for spaCy 3.8+
nlp = spacy.blank("en")         
ner = nlp.add_pipe("ner")       

# Add custom labels
for label in CUSTOM_LABELS:
    ner.add_label(label)

# 5. Training loop
n_iter = 35
optimizer = nlp.initialize()

for epoch in range(n_iter):
    random.shuffle(training_examples)
    losses = {}

    batches = minibatch(training_examples, size=compounding(4.0, 32.0, 1.5))

    for batch in batches:
        examples = [Example.from_dict(nlp.make_doc(text), ann) for text, ann in batch]
        nlp.update(examples, sgd=optimizer, drop=0.2, losses=losses)

    print(f"Epoch {epoch+1}/{n_iter} Loss: {losses}")

# 6. Save model
output_dir = "output_medical_ner"
nlp.to_disk(output_dir)
print("Model saved to", output_dir)

# 7. Quick sanity check
test_text = random.choice(data)["text"]
doc = nlp(test_text)

print("\nTest sentence:", test_text)
print("Predicted NER:", [(ent.text, ent.label_) for ent in doc.ents])


Loaded 83 annotated sentences
Custom NER labels: ['DISEASE', 'BODY_PART', 'FINDING', 'PROCEDURE', 'SYMPTOM']
Example: ('REASON FOR THE CONSULT:,  Nonhealing right ankle stasis ulcer.,HISTORY', {'entities': [(26, 36, 'FINDING'), (37, 48, 'BODY_PART'), (49, 61, 'DISEASE')]})
Epoch 1/35 Loss: {'ner': np.float32(776.6242)}
Epoch 2/35 Loss: {'ner': np.float32(269.1504)}
Epoch 3/35 Loss: {'ner': np.float32(131.03236)}
Epoch 4/35 Loss: {'ner': np.float32(118.2531)}
Epoch 5/35 Loss: {'ner': np.float32(100.19832)}
Epoch 6/35 Loss: {'ner': np.float32(97.127174)}
Epoch 7/35 Loss: {'ner': np.float32(82.98265)}
Epoch 8/35 Loss: {'ner': np.float32(72.80677)}
Epoch 9/35 Loss: {'ner': np.float32(69.08254)}
Epoch 10/35 Loss: {'ner': np.float32(76.261856)}
Epoch 11/35 Loss: {'ner': np.float32(84.334564)}
Epoch 12/35 Loss: {'ner': np.float32(57.126667)}
Epoch 13/35 Loss: {'ner': np.float32(90.96537)}
Epoch 14/35 Loss: {'ner': np.float32(81.351776)}
Epoch 15/35 Loss: {'ner': np.float32(70.72307)}
Epoch 16

### 4.2 Extended NER evaluation

In [10]:
# =========================================
#  Extended NER evaluation with test sample
# =========================================
import json
import spacy
from sklearn.metrics import precision_recall_fscore_support
from ner.utils import extract_spans, convert_to_labels

# 1. Load JSONL annotations for test
jsonl_path = PATH_TO_ANNOTATIONS + "test_annotation_complete.jsonl"

data = []
with open(jsonl_path, "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        data.append(obj)

print(f"Loaded {len(data)} annotated sentences")

# 2. Load Extended spaCy NER
nlp = spacy.load("output_medical_ner")

# 3. Convert annotations to character-level spans
gold_spans, pred_spans, labels = extract_spans(nlp, data)

# 4. Convert spans to entity sets for evaluation
true_labels, pred_labels = convert_to_labels(gold_spans, pred_spans)
# 5. Evaluate macro and micro F1
prec, rec, f1, _ = precision_recall_fscore_support(
    true_labels, pred_labels, average="micro", zero_division=0
)

print("\n===== Extended spaCy NER Evaluation =====")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 score:  {f1:.4f}")

prec_m, rec_m, f1_m, _ = precision_recall_fscore_support(
    true_labels, pred_labels, average="macro", zero_division=0
)

print("\nMacro Precision:", round(prec_m, 4))
print("Macro Recall:   ", round(rec_m, 4))
print("Macro F1:       ", round(f1_m, 4))

# 6. Show some error cases
print("\n===== Examples of WRONG predictions =====\n")

for i, (g, p, item) in enumerate(zip(gold_spans, pred_spans, data)):
    g_set = set(g)
    p_set = set(p)
    if g_set != p_set:
        print("Text:", item["text"])
        print("Gold:", g_set)
        print("Pred:", p_set)
        print("-" * 50)
        break

Loaded 26 annotated sentences

===== Extended spaCy NER Evaluation =====
Precision: 0.1333
Recall:    0.1333
F1 score:  0.1333

Macro Precision: 0.3333
Macro Recall:    0.0583
Macro F1:        0.0952

===== Examples of WRONG predictions =====

Text: Nausea.,3.
Gold: {(0, 6, 'SYMPTOM')}
Pred: set()
--------------------------------------------------


### Summary
- The performance is poor as the train and test data sets are not in the same category, and therefore don't share big enough NER vocabulary.

## 5. LLM-based NER classifier

The goal of this part was to see whether a large language model (LLM) could be used as an alternative NER system for my medical text, using only prompts and without training a new model. In practice, we tried several approaches, but all of them were limited by model size, hardware, or missing API access.

First, we tried a very small local LLM (about 0.5B parameters) via `llama-cpp`. we asked it to extract entities with my label set (DISEASE, SYMPTOM, BODY_PART, FINDING, PROCEDURE) and return a JSON list. The model usually failed to follow the format and, more importantly, produced almost random labels that did not make medical sense. This suggests that such a small model is not strong enough for domain-specific NER, even with an instruction-style prompt.

Second, we tried to use a slightly larger open-source LLM from Hugging Face (e.g. Qwen2.5-1.5B or Llama-3.2-1B) on my macOS machine via the `transformers` library. We implemented a function that builds a medical NER prompt and parses the model output into (phrase, label) pairs. However, even for one or two short sentences, generation on CPU/MPS took several minutes. Running this model on all test sentences to compute precision, recall and F1 would be impractically slow for this project.

Third, we looked at the spaCy-LLM integration. In theory, we could define an `llm` component in a `config.cfg` file, for example using a Llama2 model on Hugging Face, and then assemble it as a normal spaCy pipeline. In practice, this either requires access to an external API (e.g. OpenAI) or downloading and running a much larger model (such as Llama2-7B). we have no API credits available and my local hardware is not sufficient to run such a model efficiently.

Because of these limitations, we did not manage to build a fully working and scalable LLM-based NER baseline. Our “investigation” of LLM-based NER is therefore mainly conceptual: we explored how it would be prompted and integrated (with transformers or spaCy-LLM), and we observed that under my resource constraints it is not yet practical to use an LLM as a reliable, evaluated medical NER system. For the quantitative results in this project, we rely on the standard spaCy NER and my extended, manually trained medical NER model.




### Below is just an example of NER with Llama-3.2-1B-Instruct of one or two sentences. 

- It takes 20 min for two sentences. Therefore it is just a demo of how NER can be done. 
- If we have more computational resources, we will evaluate on the whole annotation test data.

In [4]:
from llama_cpp import Llama
import json, re

# 1. Load GGUF model (same as you already used)
llm = Llama(
    model_path="models/Llama-3.2-1B-Instruct-f16.gguf",
    n_ctx=2048,
    n_threads=6,
    n_gpu_layers=0     # works on Mac / Windows / Linux
)

print("Model loaded.")





llama_model_load_from_file_impl: using device Metal (Apple M1) - 5455 MiB free
llama_model_loader: loaded meta data with 31 key-value pairs and 147 tensors from models/Llama-3.2-1B-Instruct-f16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 1B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Llama-3.2
llama_model_loader: - kv   5:                         general.size_label str              = 1B
llama_model_loader: - kv   6:                            general.lic

Model loaded.


Using gguf chat template: {{- bos_token }}
{%- if custom_tools is defined %}
    {%- set tools = custom_tools %}
{%- endif %}
{%- if not tools_in_user_message is defined %}
    {%- set tools_in_user_message = true %}
{%- endif %}
{%- if not date_string is defined %}
    {%- if strftime_now is defined %}
        {%- set date_string = strftime_now("%d %b %Y") %}
    {%- else %}
        {%- set date_string = "26 Jul 2024" %}
    {%- endif %}
{%- endif %}
{%- if not tools is defined %}
    {%- set tools = none %}
{%- endif %}

{#- This block extracts the system message, so we can slot it into the right place. #}
{%- if messages[0]['role'] == 'system' %}
    {%- set system_message = messages[0]['content']|trim %}
    {%- set messages = messages[1:] %}
{%- else %}
    {%- set system_message = "" %}
{%- endif %}

{#- System message #}
{{- "<|start_header_id|>system<|end_header_id|>\n\n" }}
{%- if tools is not none %}
    {{- "Environment: ipython\n" }}
{%- endif %}
{{- "Cutting Knowledge Date

In [3]:
# 2. Test sentence
text = "She was having chest pains along with significant vomiting and diarrhea."

# 3. Minimal zero-shot NER prompt (no enhancements)
PROMPT = """
Extract medical named entities from the text.
Use only these labels: DISEASE, SYMPTOM, BODY_PART, FINDING, PROCEDURE.
Return ONLY a JSON list like this:
"entity_text_1":  "LABEL_1",
Text:
{TEXT}
"""

# 4. Run LLM
raw = llm(PROMPT.format(TEXT=text), max_tokens=256)
output = raw["choices"][0]["text"]
print("\nRaw LLM output:\n", output)

# 5. Parse JSON from model output
def parse_json(s):
    try:
        match = re.search(r"\[.*\]", s, re.S)
        if match:
            return json.loads(match.group(0))
    except:
        pass
    return []

ents = parse_json(output)

# 6. Print final extracted entities
print("\nParsed entities:")
for e in ents:
    print(e)

Llama.generate: 66 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =    5953.27 ms
llama_perf_context_print: prompt eval time =  735431.22 ms /     2 tokens (367715.61 ms per token,     0.00 tokens per second)
llama_perf_context_print:        eval time = 1204764.79 ms /   255 runs   ( 4724.57 ms per token,     0.21 tokens per second)
llama_perf_context_print:       total time = 1209872.05 ms /   257 tokens
llama_perf_context_print:    graphs reused =        247



Raw LLM output:
 BODY_PART:  "chest",
SYMPTOM:  "chest pains",
FINDING:  "vomiting", 
PROCEDURE:  "consulted", 
DISEASE:  "gastritis", 
BODY_PART:  "abdomen", 
SYMPTOM:  "diarrhea", 
FINDING:  "gastritis", 
BODY_PART:  "abdomen", 
SYMPTOM:  "diarrhea", 
FINDING:  "gastritis", 
BODY_PART:  "abdomen", 
PROCEDURE:  "gastritis", 

Here is an example json list:
"entity_text_1":  "DISEASE_1",
"entity_text_2":  "BODY_PART_2",
...
"entity_text_n":  "PROCEDURE_n",
"entity_text_p":  "SYMPTOM_p" ,
"entity_text_q":  "FINDING_q",
"entity_text_r":  "BODY_PART_r",
"entity_text_s":  "PROCEDURE_s",
"entity_text_t":  "SYMPTOM_t" ,
"entity_text_u":  "FINDING_u",
"entity_text_v":  "

Parsed entities:


## 6. How NER type information can help other NLP tasks

Even if the NER models in this project are not perfect, the extracted entity types are still useful for many other NLP applications in the clinical domain. Here I briefly summarise a few examples.

1. **Structuring free-text clinical notes**  
   Clinical documents are mostly free text. NER can identify key concepts and turn them into structured fields, for example:
   - DISEASE: diabetes mellitus, hypertension, stasis ulcer  
   - SYMPTOM: chest pain, nausea, diarrhea  
   - BODY_PART: right ankle, abdomen  
   - PROCEDURE: surgery, CT scan  
   This structured information can then be stored in an electronic health record or database and used for search, filtering, and statistics.

2. **Clinical decision support**  
   NER outputs can be used as features in decision support systems. For example:
   - Combinations of DISEASE and SYMPTOM entities can be used to estimate the risk of certain conditions.
   - DRUG and DOSAGE entities can be checked for possible drug interactions or dosing errors.
   In this way, NER acts as a bridge between narrative notes and automated clinical rules or prediction models.

3. **Document and patient-level classification**  
   NER types can also improve text classification. Instead of using only bag-of-words, we can use counts and patterns of entities:
   - Classifying documents by specialty (e.g. cardiology vs. endocrinology) based on the diseases and body parts mentioned.
   - Detecting potential adverse drug events by looking for co-occurrences of specific DRUG and SYMPTOM entities.
   At the patient level, the presence or absence of certain DISEASE entities can be used to define cohorts for research.

4. **Relation extraction and knowledge graphs**  
   NER is the first step towards relation extraction, such as:
   - SYMPTOM–DISEASE relations (e.g. chest pain → myocardial infarction ruled out)
   - DRUG–DISEASE relations (indications)  
   - DRUG–SYMPTOM relations (adverse effects)  
   Once entities are identified, these relations can be learned or manually defined, and combined into a clinical knowledge graph.

5. **Question answering and summarisation**  
   For question answering, knowing which spans are DISEASE or SYMPTOM helps focus retrieval on the relevant parts of a document. For summarisation, NER can be used to ensure that all important entities (diagnoses, symptoms, procedures, medications) appear explicitly in the final summary, even if the original note is long and repetitive.

Overall, NER type information turns unstructured clinical text into more interpretable and reusable signals. Even a relatively noisy NER system can already provide useful features for downstream tasks such as decision support, classification, relation extraction, and summarisation.
